## Импорт библиотек и данных

In [277]:
pip install vininfo[cli]

Note: you may need to restart the kernel to use updated packages.


In [278]:
import pandas as pd
import matplotlib.pyplot as plt
from vininfo import Vin
import numpy as np
import re
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [279]:
train_df = pd.read_csv('./train.csv')
test_x = pd.read_csv('./test.csv')
test_y = pd.read_csv('./sample_submission.csv')

In [280]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          432193 non-null  object 
 2   model         432113 non-null  object 
 3   trim          431899 non-null  object 
 4   body          429843 non-null  object 
 5   transmission  388775 non-null  object 
 6   vin           440236 non-null  object 
 7   state         440236 non-null  object 
 8   condition     430831 non-null  float64
 9   odometer      440167 non-null  float64
 10  color         439650 non-null  object 
 11  interior      439650 non-null  object 
 12  seller        440236 non-null  object 
 13  sellingprice  440236 non-null  int64  
 14  saledate      440236 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 50.4+ MB


In [281]:
test_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110060 entries, 0 to 110059
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          110060 non-null  int64  
 1   make          107999 non-null  object 
 2   model         107981 non-null  object 
 3   trim          107946 non-null  object 
 4   body          107466 non-null  object 
 5   transmission  97048 non-null   object 
 6   vin           110060 non-null  object 
 7   state         110060 non-null  object 
 8   condition     107681 non-null  float64
 9   odometer      110041 non-null  float64
 10  color         109902 non-null  object 
 11  interior      109902 non-null  object 
 12  seller        110060 non-null  object 
 13  saledate      110060 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 11.8+ MB


In [282]:
test_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110060 entries, 0 to 110059
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   vin           110060 non-null  object 
 1   sellingprice  110060 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.7+ MB


## Посмотрим данные

In [283]:
train_df.head()

# train_df.drop(['vin', 'seller'], inplace=True, axis=1)
# test_x.drop(['vin', 'seller'], inplace=True, axis=1)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,Ford,Edge,SEL,suv,automatic,2fmdk3jc4bba41556,md,4.2,111041.0,black,black,santander consumer,12500,Tue Jun 02 2015 02:30:00 GMT-0700 (PDT)
1,2014,Ford,Fusion,SE,Sedan,automatic,3fa6p0h75er208976,mo,3.5,31034.0,black,black,ars/avis budget group,14500,Wed Feb 25 2015 02:00:00 GMT-0800 (PST)
2,2012,Nissan,Sentra,2.0 SL,sedan,automatic,3n1ab6ap4cl698412,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,Wed Jun 10 2015 02:30:00 GMT-0700 (PDT)
3,2003,HUMMER,H2,Base,suv,automatic,5grgn23u93h101360,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,Wed Jun 17 2015 03:00:00 GMT-0700 (PDT)
4,2007,Ford,Fusion,SEL,Sedan,automatic,3fahp08z17r268380,md,2.0,127709.0,black,black,purple heart,1300,Tue Feb 03 2015 04:00:00 GMT-0800 (PST)


In [284]:
test_x.at[14116, 'saledate'] = 'Tue Jan 13 2015 09:15:00 GMT-0800 (PST)'
train_df['saledate'] = pd.to_datetime(train_df['saledate'], utc=True)
test_x['saledate'] = pd.to_datetime(test_x['saledate'], utc=True)

In [285]:
object_cols = train_df.select_dtypes(include='object').columns.tolist()

print(object_cols)

for item in object_cols:
    if train_df[item].dtype == 'object':
        train_df[item] = train_df[item].str.lower()
        test_x[item] = test_x[item].str.lower()


['make', 'model', 'trim', 'body', 'transmission', 'vin', 'state', 'color', 'interior', 'seller']


In [286]:
def fill_make(row):
    make = row['make']
    vin = row['vin']
    if pd.isna(make):
        vin = Vin(vin)
        return vin.manufacturer
    else:
        return make

train_df['make'] = train_df.apply(fill_make, axis=1)
test_x['make'] = test_x.apply(fill_make, axis=1)


In [ ]:
# display(train_df.groupby('model').head())
grouped_data = train_df.copy()
grouped_data = grouped_data.fillna('unknown')
grouped_data = grouped_data.groupby('make')

res_model_group = pd.DataFrame(columns=['trim', 'body', 'transmission', 'model'])

for name, group in grouped_data:
    body = group['body'].value_counts().idxmax()
    trim = group['trim'].value_counts().idxmax()
    transmission = group['transmission'].value_counts().idxmax()
    model = group['model'].value_counts().idxmax()


    result_row = pd.DataFrame({
        'make': [name],
        'body': [body],
        'trim': [trim],
        'transmission': [transmission],
        'model': [model]
    })

    res_model_group = res_model_group.append(result_row)

In [288]:
res_model_group

,trim,body,transmission,model,make
0,unknown,unknown,automatic,unknown,AM
0,unknown,unknown,automatic,unknown,Acura
0,unknown,unknown,automatic,unknown,Audi
0,unknown,unknown,automatic,unknown,BMW
0,unknown,unknown,automatic,unknown,BMW M
...,...,...,...,...,...
0,base,sedan,automatic,model s,tesla
0,le,sedan,automatic,camry,toyota
0,se pzev,sedan,automatic,jetta,volkswagen
0,t5,sedan,automatic,s60,volvo


In [289]:
def replace_missing_values(df1, df2):
    merged_df = pd.merge(df1, df2, on='make', how='left')

    merged_df['body'] = merged_df['body_x'].fillna(merged_df['body_y'])
    merged_df['trim'] = merged_df['trim_x'].fillna(merged_df['trim_y'])
    merged_df['model'] = merged_df['model_x'].fillna(merged_df['model_y'])
    merged_df['transmission'] = merged_df['transmission_x'].fillna(merged_df['transmission_y'])
    merged_df.drop(['body_x', 'body_y', 'trim_x', 'trim_y', 'transmission_x', 'transmission_y', 'model_x', 'model_y'], axis=1, inplace=True)

    return merged_df

In [290]:
train_df = replace_missing_values(train_df, res_model_group)
test_x = replace_missing_values(test_x, res_model_group)


In [292]:
display(train_df.info())
display(test_x.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   year          440236 non-null  int64              
 1   make          440236 non-null  object             
 2   vin           440236 non-null  object             
 3   state         440236 non-null  object             
 4   condition     430831 non-null  float64            
 5   odometer      440167 non-null  float64            
 6   color         439650 non-null  object             
 7   interior      439650 non-null  object             
 8   seller        440236 non-null  object             
 9   sellingprice  440236 non-null  int64              
 10  saledate      440236 non-null  datetime64[ns, UTC]
 11  body          440236 non-null  object             
 12  trim          440236 non-null  object             
 13  model         440236 non-null  object       

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110060 entries, 0 to 110059
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype              
---  ------        --------------   -----              
 0   year          110060 non-null  int64              
 1   make          110060 non-null  object             
 2   vin           110060 non-null  object             
 3   state         110060 non-null  object             
 4   condition     107681 non-null  float64            
 5   odometer      110041 non-null  float64            
 6   color         109902 non-null  object             
 7   interior      109902 non-null  object             
 8   seller        110060 non-null  object             
 9   saledate      110060 non-null  datetime64[ns, UTC]
 10  body          110056 non-null  object             
 11  trim          110059 non-null  object             
 12  model         110059 non-null  object             
 13  transmission  110059 non-null  object       

None

In [293]:
column_for_fill = ['trim', 'body', 'model', 'transmission']
test_x[column_for_fill] = test_x[column_for_fill].fillna('unknown')

In [298]:
for item in object_cols:
    if train_df[item].dtype == 'object':
        print(item)
        print(train_df[item].unique())

make
['ford' 'nissan' 'hummer' 'lincoln' 'pontiac' 'hyundai' 'Toyota' 'buick'
 'chevrolet' 'honda' 'acura' 'cadillac' 'gmc' 'saab' 'dodge'
 'mercedes-benz' 'toyota' 'volkswagen' 'Mazda' 'bmw' 'infiniti' 'chrysler'
 'kia' 'Ford' 'Chevrolet USA' 'jaguar' 'subaru' 'jeep' 'lexus' 'mercedes'
 'scion' 'Chevrolet Mexico' 'fiat' 'suzuki' 'Land Rover' 'mazda' 'volvo'
 'audi' 'CAMI' 'mini' 'isuzu' 'GMC Truck' 'mitsubishi' 'Chrysler Canada'
 'Jeep' 'smart' 'porsche' 'Dodge' 'land rover' 'saturn' 'Hyundai' 'ram'
 'DaimlerChrysler AG/Daimler AG' 'oldsmobile' 'Chevrolet' 'Dodge Canada'
 'mercury' 'Porsche car' 'bentley' 'Mitsubishi' 'Mercedes-Benz' 'Cadillac'
 'Scion' 'Volkswagen' 'Mercury' 'BMW' 'Chrysler' 'fisker' 'Lexus' 'Honda'
 'Audi' 'Nissan' 'Dodge Mexico' 'maserati' 'Lincoln' 'Genesis' 'Pontiac'
 'tesla' 'landrover' 'vw' 'Daimler AG (Sprinter)' 'AM' 'Acura' 'GMC'
 'gmc truck' 'Chevrolet Canada' 'rolls-royce' 'ferrari' 'plymouth'
 'lamborghini' 'Maserati' 'Suzuki' 'Smart' 'Kia' 'UnsupportedBr

In [302]:
train_df['transmission'] = train_df['transmission'].replace('unknown', train_df['transmission'].mode()[0])
test_x['transmission'] = test_x['transmission'].replace('unknown', test_x['transmission'].mode()[0])


In [ ]:
train_df.columns[train_df.isna().any()].tolist()

['condition', 'odometer', 'color', 'interior']

In [296]:
test_x.columns[test_x.isna().any()].tolist()

['condition', 'odometer', 'color', 'interior']

In [ ]:
train_df.loc[train_df['make'].isna()].head()

,year,make,vin,state,condition,odometer,color,interior,seller,saledate,body,trim,model,transmission


In [ ]:
test_x.loc[test_x['make'].isna()]


,year,make,vin,state,condition,odometer,color,interior,seller,saledate,body,trim,model,transmission


In [ ]:
vin.verify_checksum()

True

## Работа с датой

## Работа с пропусками

## Работа с дубликатами

## Работа с выбросами